# Exercise 5: Production Schedule

Problem statement:

## Multi-Product Manufacturing with Setup Costs
A specialty electronics manufacturer produces 4 different circuit board types on
3 production lines over a 5-day planning horizon. They want to minimize total costs (production + setup).

### Production lines:

Line A: High precision, $50/hour, 40 hours available per day
Line B: Standard quality, $35/hour, 48 hours available per day
Line C: High volume, $25/hour, 60 hours available per day

### Products & requirements:

Premium boards: 2 hours/unit, $200 profit each, can only use Line A
Standard boards: 1.5 hours/unit, $120 profit each, can use Line A or B
Basic boards: 1 hour/unit, $80 profit each, can use any line
Prototype boards: 4 hours/unit, $500 profit each, can only use Line A

### Setup costs
There are one-time costs if you produce that product on that line.

* Premium on Line A: $2,000 setup
* Standard on Line A: $1,500 setup, on Line B: $1,000 setup
* Basic on any line: $500 setup
* Prototype on Line A: $3,000 setup

### Customer demands
(minimum quantities needed by end of week):

* Premium: 50 units
* Standard: 80 units
* Basic: 200 units
* Prototype: 15 units

Business rule: Once you start production of a product on a line, you must produce at least 10 units (to justify the setup cost).

Goal: Maximize profit (revenue - production costs - setup costs) while meeting all demands.

## Modelling production lines over time

Let's focus on modelling a single production line, because once we figure that out we can expand that approach to multiple lines.

Modelling the time for a single line, seems there are two major ways we might do this. Producing items takes a variety of times including 1 hour, 2 hours, and 1.5 hours.

* **Periods**. We might model time as half-hour chunks, and within each chunk make *N* boolean variables that says whether the line makes each different item.  We'd make a constraint that says that we are only making a single item at a time.  But we'd also need to make a series of constraints that say that once we start making an item, we have to keep making that item for a certain number of periods until it's done.  This would get challenging to make sure there are no gaps.

* **Production List.** We could model the first time *t* subscript as the *first item* that the line produces, and the second *t* subscript as the *second item* that the line produces.  A given subscript of *t* could represent an hour if the item it's producing takes an hour, and the second subscript might represent 1.5 hours if the second item takes 1.5 hours.  This will make it much easier to represent the constraints if we take this approach.

## Simple demonstration

Let's try a simple demonstration of the idea with 1 production line and 2 products to demonstrate the idea and work out the basics.

For this problem let's say:

1. We have 1 production line
2. We have 2 products:
    * Lamps take 1 hour to make and produce 20 profit.
    * Timers take 1.5 hours to make and produce 40 profit.
3. We must produce at least 2 lamps and 2 timers.
4. We have a single shift of 12 hours.
5. Maximize profit.

This is a super simple example but will help us figure out the modelling strategy.

In [1]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

In [2]:
all_items = ['lamp', 'timer']
profit_by_item = {'lamp': 20, 'timer': 40}
periods_by_item = {'lamp': 2, 'timer': 3} # half-hour periods.
min_demand_by_item = {'lamp': 2, 'timer': 2}
shift_max_periods = 24 # 12 hour total run time max.
max_slots = 12 # A "slot" here means a thing that we made on the line.
all_slots = list(range(max_slots)) # Slot 0 is the first thing we made, slot 1 is the second

In [3]:
# Decision variables
make_vars = {} # Key: (slot, item) - true if we're making this item on the line during this slot.
for slot in all_slots:
    for item in all_items:
        make_vars[slot, item] = solver.BoolVar(f'make_{slot}_{item}')

In [4]:
# Constraint: can only make one item at a time.
for slot in all_slots:
    constraint = solver.Constraint(0, 1, 'slot_{slot}_item') # Up to 1 item can be selected.
    for item in all_items:
        constraint.SetCoefficient(make_vars[slot, item], 1)

In [5]:
# Constraint: minimum production quantities
infinity = solver.infinity()
for item in all_items:
    constraint = solver.Constraint(min_demand_by_item[item], infinity, f'min_demand_{item}')
    for slot in all_slots:
        constraint.SetCoefficient(make_vars[slot, item], 1)

In [6]:
# Constraint: total number of periods used must be <= shift_max_periods (12 hour day)
constraint = solver.Constraint(0, shift_max_periods, f'shift_max_periods')
for item in all_items:
    for slot in all_slots:
        # If we're making this item in this slot, this is how many periods that will take:
        constraint.SetCoefficient(make_vars[slot, item], periods_by_item[item])

In [7]:
# Objective: Maximize profit
objective = solver.Objective()
for slot in all_slots:
    for item in all_items:
        objective.SetCoefficient(make_vars[slot, item], profit_by_item[item])
objective.SetMaximization()

In [ ]:
# Solve
result_status = solver.Solve()

In [9]:
is_solved = result_status == pywraplp.Solver.OPTIMAL
if is_solved:
    print('Solved!')
else:
    print('NOT Solved')

Solved!


In [19]:
print(f'Objective value: profit={objective.Value()}')
for slot in all_slots:
    selected_items = [item for item in all_items if make_vars[slot, item].solution_value()]
    if not selected_items:
        continue
    if len(selected_items) > 1:
        print('This should not happen! Have more than one item to be made during this slot')
    item = selected_items[0]
    print(f'{slot+1}. {item}')

Objective value: profit=300.0
1. timer
2. timer
3. timer
4. timer
5. timer
6. timer
7. lamp
8. lamp
9. lamp


So we'll be making 6 timers and 3 lamps. This comes out to exactly the duration of our shift.